# Communicating Sequential Processes Paradigm

## Initial Set Up

In [3]:
import gevent.queue as gq

In [1]:
import gevent

In [33]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Callable, Any, Union
from gevent.greenlet import Greenlet

@dataclass
class Process:
    function: Callable[..., Any]
    use_sink: bool = False
    inlet: gq.Queue = field(init=False)
    outlet: gq.Queue = field(init=False)

    def __post_init__(self):
        self.inlet = gq.Queue()
        self.outlet = gq.Queue()

    @staticmethod
    def has_more_messages(queue: gq.Queue, timeout: int = 2):
        for _ in range(timeout * 10):
            if not queue.empty():
                return True
            gevent.sleep(0.1)
        return False


    def _run(self):
        while has_more_messages(self.inlet):
            arguments = self.inlet.get()
            result = self.function(arguments)
            self.outlet.put(result)


    def run(self) -> Greenlet:
        return gevent.spawn(self._run)

    def read_from(self, other: Process):
        self.inlet = other.outlet

    def write_to(self, other: Process):
        other.inlet = self.outlet


In [34]:
def identity(x):
    return x

In [36]:
reader_process = Process(function=print)
writer_process = Process(function=identity)

reader_process.read_from(writer_process)

def trigger_process(process: Process, inputs: Any):
    def worker():
        for input_ in inputs:
            process.inlet.put(input_)
    return worker

gevent.joinall([
    gevent.spawn(trigger_process(writer_process, [1, 2, 3])),
    writer_process.run(),
    reader_process.run(),
])

1
2
3


[<Greenlet at 0x20fdfd4d800: _run>,
 <Greenlet at 0x20fdfd4d940: _run>,
 <Greenlet at 0x20fdfd4d9e0: _run>]

## Predecessor and Successor

In [1]:
def predecessor(a: int) -> int:
    return a - 1


assert predecessor(1) == 0
assert predecessor(10) == 9


In [2]:
def successor(a: int) -> int:
    return a + 1


assert successor(0) == 1
assert successor(10) == 11


## Addition

In [3]:
def addition(addend_1: int, addend_2: int) -> int:
    result = addend_1
    for _ in range(addend_2):
        result = successor(result)
    return result


assert addition(0, 0) == 0
assert addition(1, 0) == 1
assert addition(0, 1) == 1
assert addition(10, 10) == 20


## Multiplication

In [4]:
def multiplication(multiplicand: int, multiplier: int) -> int:
    if multiplicand == 0 or multiplier == 0:
        return 0

    result = 0
    for _ in range(multiplier):
        result = addition(result, multiplicand)
    return result


assert multiplication(0, 0) == 0
assert multiplication(2, 0) == 0
assert multiplication(0, 2) == 0
assert multiplication(10, 10) == 100


## Exponentiation

In [5]:
def exponentiation(base: int, exponent: int) -> float:
    result = 1
    for _ in range(exponent):
        result = multiplication(result, base)
    return result


assert exponentiation(1, 0) == 1
assert exponentiation(0, 1) == 0
assert exponentiation(3, 3) == 27
